In [1]:
!pip install --upgrade --force-reinstall protobuf==3.20.3

import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow. keras.regularizers import l2

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-me

2025-11-20 06:20:31.652889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763619631.815005      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763619631.863187      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
base_dir = '/kaggle/input/skin-ds'
train = os.path.join(base_dir, "train")
test = os.path.join(base_dir, "test")
val = os.path.join(base_dir, "val")

img_size = (224,224)
batch_size = 64

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True

)

test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    directory = train,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory = test,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    directory = val,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

print("Class Mappings Train:", train_generator.class_indices)

Found 29322 images belonging to 14 classes.
Found 3674 images belonging to 14 classes.
Found 3660 images belonging to 14 classes.
Class Mappings Train: {'Actinic keratoses': 0, 'Basal cell carcinoma': 1, 'Benign keratosis-like lesions': 2, 'Chickenpox': 3, 'Cowpox': 4, 'Dermatofibroma': 5, 'HFMD': 6, 'Healthy': 7, 'Measles': 8, 'Melanocytic nevi': 9, 'Melanoma': 10, 'Monkeypox': 11, 'Squamous cell carcinoma': 12, 'Vascular lesions': 13}


In [9]:
model = Sequential([
        Conv2D(filters=32, kernel_size=(5, 5), padding='valid', input_shape=(224, 224, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
    
        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
            Activation('relu'),
            MaxPooling2D(pool_size=(2, 2)),
            BatchNormalization(),
        
        Conv2D(filters=128, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Flatten(),
        
        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=14, activation='softmax')
    ])

In [4]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)


In [5]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [6]:
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])


In [7]:
history = model.fit(train_generator, epochs=50, validation_data=val_generator,
                       verbose=2,
                       callbacks=[reduce_lr, early_stopping])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1763619690.486071     132 service.cc:148] XLA service 0x7c4b04010c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763619690.486728     132 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1763619690.923499     132 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1763619697.842916     132 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


459/459 - 790s - 2s/step - accuracy: 0.3172 - loss: 9.2268 - val_accuracy: 0.3970 - val_loss: 1.8451 - learning_rate: 0.0010
Epoch 2/50
459/459 - 505s - 1s/step - accuracy: 0.3836 - loss: 1.9587 - val_accuracy: 0.4216 - val_loss: 1.6667 - learning_rate: 0.0010
Epoch 3/50
459/459 - 495s - 1s/step - accuracy: 0.3943 - loss: 1.8478 - val_accuracy: 0.4350 - val_loss: 1.6074 - learning_rate: 0.0010
Epoch 4/50
459/459 - 498s - 1s/step - accuracy: 0.4039 - loss: 1.7755 - val_accuracy: 0.4508 - val_loss: 1.5288 - learning_rate: 0.0010
Epoch 5/50
459/459 - 499s - 1s/step - accuracy: 0.4106 - loss: 1.7485 - val_accuracy: 0.4388 - val_loss: 1.5835 - learning_rate: 0.0010
Epoch 6/50
459/459 - 502s - 1s/step - accuracy: 0.4161 - loss: 1.7142 - val_accuracy: 0.4369 - val_loss: 1.5851 - learning_rate: 0.0010
Epoch 7/50
459/459 - 504s - 1s/step - accuracy: 0.4104 - loss: 1.7111 - val_accuracy: 0.4180 - val_loss: 1.7920 - learning_rate: 0.0010
Epoch 8/50
459/459 - 503s - 1s/step - accuracy: 0.4138 - lo